# Extract Trending Keywords

여러 프로필에서 핫 키워드 추출하기

In [1]:
from typing import List, Dict
from collections import Counter
import random

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

/Users/hyeonjinho/.pyenv/versions/3.11.6/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


## Keyword Extraction Chain 만들기

In [2]:
model_3_5 = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.8)
model = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0.8)

In [3]:
class Keywords(BaseModel):
    keyword_list: List[str] = Field(description="List of keywords")

In [4]:
parser = JsonOutputParser(pydantic_object=Keywords)
format_instructions = parser.get_format_instructions()
format_instructions

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"keyword_list": {"title": "Keyword List", "description": "List of keywords", "type": "array", "items": {"type": "string"}}}, "required": ["keyword_list"]}\n```'

In [5]:
human_prompt_template = HumanMessagePromptTemplate.from_template(
                            "{doc}\n{format_instructions}")

prompt = ChatPromptTemplate.from_messages(
    [
        human_prompt_template,
    ])
prompt = prompt.partial(format_instructions=format_instructions)

In [6]:
extract_keyword_chain = prompt | model | parser

In [7]:
profile = model.invoke("20대 평범한 직장인 자기소개 하나 만들어줘").content
print(profile)

안녕하세요! 저는 20대 후반의 평범한 직장인입니다. 현재 IT업계에서 마케팅 담당자로 근무하고 있어요. 대학에서 경영학을 전공하고, 졸업 후 이 분야에서 제 커리어를 시작했습니다. 

일하는 것 외에도 여러 취미가 있는데, 특히 음악 감상과 요리하기를 좋아합니다. 주말이면 새로운 레시피를 시도하는 것이 제 소소한 즐거움 중 하나예요. 또한, 운동을 꾸준히 하려고 노력 중이며, 요즘엔 특히 요가에 관심이 많습니다.

사람들과 소통하는 것을 좋아하고, 다양한 경험을 통해 새로운 것을 배우는 것을 두려워하지 않습니다. 앞으로도 다양한 분야에 도전하며, 제 역량을 넓혀나가고자 합니다. 

이렇게 저는 평범하지만, 제 삶을 즐기며 살아가는 20대 직장인입니다. 앞으로도 많은 성장과 발전을 이루어 나가고 싶습니다. 감사합니다!


In [8]:
extract_keyword_chain.invoke({'doc': profile})

{'keyword_list': ['20대 후반',
  '직장인',
  'IT업계',
  '마케팅',
  '경영학 전공',
  '음악 감상',
  '요리',
  '주말',
  '새로운 레시피',
  '운동',
  '요가',
  '소통',
  '다양한 경험',
  '도전',
  '역량 넓히기',
  '성장',
  '발전']}

## Trending Keywords 추출하기

1. pseudo profile description 만들기
2. description으로 profile sampling 하기
3. Trending Keyword 추출


In [9]:
# 직업, 연령대, 그리고 추가 특징 리스트 정의
jobs = ["금융 분석가", "소프트웨어 엔지니어", "창업가", "그래픽 디자이너", "초등학교 교사", "마케팅 전문가", "건축가", "의사", "간호사", "운동선수"]
age_groups = ["20대 초반", "20대 후반", "30대 초반", "30대 중반", "40대 초반", "40대 중반"]
features = ["자기소개 하나 만들어줘", "에 대한 간략한 소개 부탁해", "의 일상과 업무에 대해 알려줘"]

# 설명 생성 함수
def create_description() -> str:
    job = random.choice(jobs)
    age_group = random.choice(age_groups)
    feature = random.choice(features)
    description = f"{age_group}의 {job} {feature}"
    return description

# 여러 가상의 설명 생성
descriptions = [create_description() for _ in range(20)]

# 생성된 설명 확인
for description in descriptions:
    print(description)


20대 초반의 건축가 에 대한 간략한 소개 부탁해
30대 중반의 초등학교 교사 의 일상과 업무에 대해 알려줘
30대 중반의 운동선수 자기소개 하나 만들어줘
40대 초반의 의사 자기소개 하나 만들어줘
20대 초반의 소프트웨어 엔지니어 에 대한 간략한 소개 부탁해
30대 초반의 운동선수 의 일상과 업무에 대해 알려줘
30대 중반의 창업가 에 대한 간략한 소개 부탁해
30대 중반의 소프트웨어 엔지니어 의 일상과 업무에 대해 알려줘
40대 초반의 소프트웨어 엔지니어 자기소개 하나 만들어줘
40대 중반의 의사 에 대한 간략한 소개 부탁해
30대 초반의 마케팅 전문가 자기소개 하나 만들어줘
40대 중반의 마케팅 전문가 에 대한 간략한 소개 부탁해
40대 중반의 금융 분석가 의 일상과 업무에 대해 알려줘
20대 초반의 금융 분석가 자기소개 하나 만들어줘
20대 후반의 운동선수 자기소개 하나 만들어줘
20대 초반의 의사 에 대한 간략한 소개 부탁해
30대 중반의 소프트웨어 엔지니어 에 대한 간략한 소개 부탁해
40대 중반의 운동선수 에 대한 간략한 소개 부탁해
30대 중반의 초등학교 교사 자기소개 하나 만들어줘
30대 초반의 건축가 의 일상과 업무에 대해 알려줘


In [10]:
# 프로필 생성을 위한 함수 정의
def create_profile(description: str) -> str:
    response = model_3_5.invoke(description).content
    return response

# 각 설명에 대해 프로필 생성
profiles = [create_profile(description) for description in descriptions]

# 생성된 프로필 확인
for i, profile in enumerate(profiles, start=1):
    print(f"Profile {i}:\n{profile}\n")

Profile 1:
20대 초반의 건축가는 전문적인 건축 교육을 받고 현장 경험이 부족한 신진 건축가로, 건축 디자인에 대한 열정과 창의성을 가지고 있습니다. 이들은 새로운 아이디어와 기술을 통해 현대적이고 혁신적인 건축물을 설계하며, 다양한 프로젝트를 경험하며 자신의 스타일과 방향성을 찾아가고 있습니다. 또한, 끊임없는 자기계발과 학습을 통해 전문성을 향상시키고, 건축계에서 인정받을 수 있는 전문가로 성장하고자 합니다.

Profile 2:
30대 중반의 초등학교 교사의 일상과 업무는 매우 바쁜 일정을 갖고 있습니다. 보통 아침 일찍 출근하여 학교 준비를 하고, 학생들을 맞이하며 수업을 진행합니다. 

수업이 끝나면 학생들의 안전을 지키기 위해 교실 정리 및 방청활동, 보호자와의 소통 등 다양한 업무를 처리하게 됩니다. 또한 학생들의 학업 및 생활에 대한 관리와 지도를 위해 학습지도 및 학생 상담 업무도 수행합니다.

또한 교과서의 준비와 교육과정에 따른 교육자료 제작, 학급운영 계획 수립 및 평가 등 교사로서의 전문적인 업무도 맡게 됩니다. 또한 학부모와의 소통을 통해 학생들의 학업 및 학교생활에 대한 정보를 공유하고 협력하여 학생들의 발전을 도모하는 역할도 중요한 부분입니다. 

또한 학생들의 안전과 행복을 지키기 위해 학교 생활 규칙 및 안전 프로토콜을 준수하고, 학생들의 안전 및 보호를 위한 대처 및 상황 관리 역시 교사의 책임이자 업무 중 하나입니다. 

이렇게 바쁜 업무와 책임을 맡고 있지만, 학생들과의 소통과 교육에 대한 열정으로 인해 행복한 순간들을 경험하며, 학생들의 성장과 발전을 도와주는 데 큰 보람을 느낄 수 있습니다.

Profile 3:
안녕하세요! 저는 30대 중반의 운동선수 XXX입니다.

저는 어릴 때부터 운동을 사랑하고, 그로 인해 전국 대회에서 수상한 경력을 가지고 있습니다. 현재는 주로 농구와 테니스를 즐기고 있습니다. 농구를 통해 스피드와 슈팅 능력을 향상시키고, 테니스를 통해 근력과 집중력을 키우고 있습니다.

운동을 통

In [11]:
# 이미 정의된 모델과 파서를 사용하여 여러 프로필로부터 키워드를 추출하는 함수
def extract_keywords_from_profiles(profiles: List[str]) -> List[str]:
    keywords = []
    for profile in profiles:
        result = extract_keyword_chain.invoke({'doc': profile})['keyword_list']
        keywords.extend(result)
    return keywords

In [12]:
# 여러 프로필에서 키워드 추출
extracted_keywords = extract_keywords_from_profiles(profiles)
extracted_keywords

['early-career architect',
 'professional architectural education',
 'lack of field experience',
 'passionate about architectural design',
 'creativity',
 'innovative architecture',
 'contemporary design',
 'exploring personal style',
 'diverse project experience',
 'continuous self-improvement',
 'enhancing expertise',
 'aspiring to be recognized in the architecture field',
 '출근',
 '학교 준비',
 '수업 진행',
 '학생 안전',
 '교실 정리',
 '보호자 소통',
 '학습지도',
 '학생 상담',
 '교과서 준비',
 '교육자료 제작',
 '학급운영 계획',
 '학부모 소통',
 '학생 발전',
 '학교 생활 규칙',
 '안전 프로토콜',
 '상황 관리',
 '교육 열정',
 '학생 성장',
 '운동선수',
 '30대',
 '전국 대회',
 '수상',
 '농구',
 '테니스',
 '스피드',
 '슈팅',
 '근력',
 '집중력',
 '건강',
 '활기찬 삶',
 '자신감',
 '목표',
 '성장',
 '성과',
 '동료',
 '노력',
 '통합 의학',
 '만성질환',
 '스트레스',
 '정신건강',
 '소통',
 '환자 의견 쭂',
 '건강과 행복',
 '상담',
 '소프트웨어 엔지니어',
 '프로그래밍',
 '컴퓨터 공학',
 '소프트웨어 개발',
 '웹 개발',
 '프론트엔드',
 '백엔드',
 '새로운 기술',
 '문제 해결',
 '협업',
 '공부',
 '성장',
 '운동선수',
 '아침 일찍',
 '조깅',
 '요가',
 '스트레칭',
 '식사',
 '훈련',
 '체력',
 '근력',
 '영양가 있는 식사',
 '영양 제품 복용',
 '마사지'

In [13]:
# 키워드의 빈도를 계산하고 가장 인기 있는 핫 키워드를 추출하는 함수
def get_top_keywords(keywords: List[str], top_n: int = 5) -> List[Dict[str, int]]:
    keyword_frequency = Counter(keywords)
    most_common_keywords = keyword_frequency.most_common(top_n)
    return most_common_keywords

In [14]:
# 인기 키워드 추출
top_keywords = get_top_keywords(extracted_keywords, top_n=5)
top_keywords

print(top_keywords)

[('근력', 4), ('성장', 4), ('소프트웨어 엔지니어', 4), ('운동선수', 3), ('소프트웨어 개발', 3)]
